In [171]:
import pandas as pd 
import numpy as np 
from tqdm import tqdm
import re

import warnings
warnings.filterwarnings("ignore")

MAX_KEY_SIZE = 26

In [172]:
def load_dataset(dataset_path):
    np_dataset = np.genfromtxt(dataset_path, dtype=str)
    np_password = np.ndarray(len(np_dataset), dtype="object")
    np_user = np.ndarray(len(np_dataset), dtype="object")
    return np_dataset, np_password, np_user

In [173]:
def format_dataset(dataset: np.ndarray):
    pattern = "\|(.*?)\|"
    for i in tqdm(range(dataset.shape[0])):
        np_password[i] = re.search(pattern, np_dataset[i][1:]).group(1)
        np_user[i] = np_dataset[i].replace(np_password[i], "").replace(".", " ").replace("|", "")

In [174]:
def getMode():
    while 1:
        print('Do you want to encrypt or decrypt your database?')
        mode = input().lower()
        if mode in 'encrypt e decrypt d'.split():
            return mode
        else:
            print('You can type encrypt, e, decrypt or d only')

In [175]:
def getData():
    key = ''
    while 1:
        print('Do you want to encrypt/decrypt only the password or the entire base?')
        data = input().lower()
        if data in 'password p entire base base b'.split():
            return data
        else:
            print('You can type password, p, entire, base or b only')

In [176]:
def getKey():
    key = 0
    while True:
        print('Type a number between 1 and %s' % (MAX_KEY_SIZE))
        key = int(input())
        if (key >= 1 and key <= MAX_KEY_SIZE):
            return key


In [177]:
def getTranslatedMessage(mode, message, key):
    if mode[0] == 'd':
        key = -key
    translated = ''

    for symbol in message:
        if symbol.isalpha():
            num = ord(symbol)
            num += key

            if symbol.isupper():
                if num > ord('Z'):
                    num -= 26
                elif num < ord('A'):
                    num += 26
            elif symbol.islower():
                if num > ord('z'):
                    num -= 26
                elif num < ord('a'):
                    num += 26

            translated += chr(num)
        else:
            translated += symbol
    return translated

In [178]:
crypt_or_auth = str(input("Do you want to validate an user or encrypt your dataset? Please, type validate to validate an user or encrypt to encrypt you dataset"))

if(crypt_or_auth == "validate"):
    user = str(input("Please, type the password and the key you'd used to encrypt your database (for no encrypted database type 0)")).split()
    pswrd = user[0]
    key = user[1]
    
    if key == '0':
        np_dataset, np_password, np_user = load_dataset(r'Datasets\base.txt')
        format_dataset(np_dataset)
        for i in tqdm(range(len(np_password))):
            if np_password[i] == pswrd:
                print("\n")
                print("User validated")
    
    else:
        pdst = pd.read_csv(r'Datasets\pandas.txt', header=None)
        pdst.columns = ['User', 'Password']
        pswrd = getTranslatedMessage('encrypt', pswrd, int(user[1]))
        for i in tqdm(range(pdst['Password'].shape[0])):
            if pdst['Password'][i] == pswrd:
                print("\n")
                print("User validated")

else:
    opt = str(input("Please, type base to select the uncrypted dataset or encrypted to select the encrypted dataset: "))
    if(opt == "base"):
        np_dataset, np_password, np_user = load_dataset(r'Datasets\base.txt')
        format_dataset(np_dataset)
    else:
        np_dataset, np_password, np_user = load_dataset(r'Datasets\pandas.txt')
        format_dataset(np_dataset)
        
    mode = getMode()
    data = getData()
    key = getKey()

    mode_crypt = ''    
    for i in tqdm(range(np_dataset.shape[0])):
        if data in 'password p'.split():
            mode_crypt = 'Password'
            np_password[i] = getTranslatedMessage(mode, np_password[i], key)
        else:
            np_user[i] = getTranslatedMessage(mode, np_user[i], key)
            np_password[i] = getTranslatedMessage(mode, np_password[i], key)

Do you want to validate an user or encrypt your dataset? Please, type validate to validate an user or encrypt to encrypt you datasetvalidate
Please, type the password and the key you'd used to encrypt your database (for no encrypted database type 0)OOGJRNIN 9


100%|██████████████████████████████████████████████████████████████████████| 300000/300000 [00:01<00:00, 258866.11it/s]



User validated


In [168]:
df = pd.DataFrame(np_user)

In [169]:
df.columns = ['User']
df['Password'] = np_password

In [170]:
df.to_csv(r'Datasets\pandas.txt', header=None, index=None, sep=',', mode='w')